# In this notebook we would build the CNN model

In [9]:
"""
1-D CNN for BTC next-hour direction
----------------------------------
* quick 5-trial hyper-parameter search (val Weighted-F1, precision×2)
* full training with the best config
* prints per-class metrics

pip install pandas numpy scikit-learn tensorflow keras-tuner
"""

import os, shutil, numpy as np, pandas as pd, tensorflow as tf, keras_tuner as kt
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

# ───── Paths & params ───────────────────────────────────────────────
CSV_PATH  = r"C:\Users\ADMIN\Desktop\Coding_projects\stock_market_prediction\Stock-Market-Prediction\data\processed\gemini_btc_data_final_version_with_features_2016_final.csv"
DROP_COLS = ["vol_ratio_24h", "macd_diff", "macd_line", "upper_shadow", "lower_shadow"]

SEQ_LEN   = 60
VAL_FRAC  = 0.20
W_PREC    = 2.0           # precision weight in F-score
SEARCH_TRIALS = 5         # keep tiny → fast (<5 min on CPU)
SEARCH_EPOCHS = 3
FULL_EPOCHS   = 30
BATCH         = 64

# ───── Weighted-F1 metric (precision×2) ─────────────────────────────
class WeightedF1(tf.keras.metrics.Metric):
    def __init__(self, weight=2.0, name="weighted_f1", threshold=0.5, **kw):
        super().__init__(name=name, **kw)
        self.w  = weight
        self.th = threshold
        self.tp = self.add_weight(name="tp", initializer="zeros")
        self.fp = self.add_weight(name="fp", initializer="zeros")
        self.fn = self.add_weight(name="fn", initializer="zeros")

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_pred = tf.cast(y_pred >= self.th, tf.float32)
        y_true = tf.cast(y_true, tf.float32)
        self.tp.assign_add(tf.reduce_sum(y_true * y_pred))
        self.fp.assign_add(tf.reduce_sum((1 - y_true) * y_pred))
        self.fn.assign_add(tf.reduce_sum(y_true * (1 - y_pred)))

    def result(self):
        prec = self.tp / (self.tp + self.fp + 1e-7)
        rec  = self.tp / (self.tp + self.fn + 1e-7)
        return (1 + self.w) * prec * rec / (self.w * prec + rec + 1e-7)

    def reset_states(self):
        for v in (self.tp, self.fp, self.fn):
            v.assign(0.)

# ───── Data prep (identical to previous code) ───────────────────────
df = pd.read_csv(CSV_PATH, index_col=0, parse_dates=True)
df = df.drop(columns=[c for c in DROP_COLS if c in df.columns])
df["Volume BTC"] = np.log1p(df["Volume BTC"])
df["target"] = (df["close"].shift(-1) > df["close"]).astype(int)
df = df.dropna().select_dtypes(include=[np.number])

feature_cols = df.columns.drop("target")
split_raw = int(len(df) * (1 - VAL_FRAC))
train_raw, val_raw = df.iloc[:split_raw], df.iloc[split_raw:]

scaler = StandardScaler().fit(train_raw[feature_cols])
df_scaled = pd.DataFrame(
    scaler.transform(df[feature_cols]),
    columns=feature_cols, index=df.index
)
labels = df["target"].values.astype(np.float32)

def make_sequences(mat, tgt, length):
    Xs, ys = [], []
    for i in range(length, len(mat)):
        Xs.append(mat[i-length:i])
        ys.append(tgt[i])
    return np.array(Xs, dtype=np.float32), np.array(ys, dtype=np.float32)

X_all, y_all = make_sequences(df_scaled.values, labels, SEQ_LEN)
split_seq = int(len(X_all) * (1 - VAL_FRAC))
X_train, X_val = X_all[:split_seq], X_all[split_seq:]
y_train, y_val = y_all[:split_seq], y_all[split_seq:]
n_features = X_train.shape[2]

# ───── Hyper-model builder ─────────────────────────────────────────-
def build_model(hp):
    m = tf.keras.Sequential()
    m.add(tf.keras.layers.Input(shape=(SEQ_LEN, n_features)))

    blocks = hp.Int("conv_blocks", 1, 2)
    for i in range(blocks):
        filters = hp.Int(f"filters_{i}", 32, 128, step=32)
        ksize   = hp.Choice(f"kernel_{i}", [3, 5, 7, 9])
        m.add(tf.keras.layers.Conv1D(filters, ksize, padding="causal", activation="relu"))
        m.add(tf.keras.layers.BatchNormalization())

    m.add(tf.keras.layers.GlobalAveragePooling1D())
    m.add(tf.keras.layers.Dropout(hp.Float("dropout", 0.0, 0.5, step=0.1)))
    m.add(tf.keras.layers.Dense(1, activation="sigmoid"))

    m.compile(
        optimizer=tf.keras.optimizers.Adam(hp.Float("lr", 1e-4, 1e-2, sampling="log")),
        loss="binary_crossentropy",
        metrics=[WeightedF1(weight=W_PREC)]
    )
    return m

# ───── Quick RandomSearch ─────────────────────────────────────────--
TMP_DIR = "tmp_cnn_tune"
shutil.rmtree(TMP_DIR, ignore_errors=True)      # clean each run

tuner = kt.RandomSearch(
    build_model,
    objective=kt.Objective("val_weighted_f1", direction="max"),
    max_trials=SEARCH_TRIALS,
    executions_per_trial=1,
    directory=TMP_DIR, project_name="run", overwrite=True
)

tuner.search(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=SEARCH_EPOCHS,
    batch_size=BATCH,
    shuffle=False,
    verbose=0
)

best_hp = tuner.get_best_hyperparameters(1)[0]

print("\n──── Best hyper-parameters ────")
for k, v in best_hp.values.items():
    print(f"{k:<12}: {v}")

# ───── Build final model with best params ─────────────────────────--
model = tuner.hypermodel.build(best_hp)

early_stop = tf.keras.callbacks.EarlyStopping(
    patience=5, restore_best_weights=True, monitor="val_loss"
)

model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=FULL_EPOCHS,
    batch_size=BATCH,
    shuffle=False,
    callbacks=[early_stop],
    verbose=2
)

# ───── Metrics ─────────────────────────────────────────────────────
y_prob = model.predict(X_val, batch_size=BATCH).flatten()
y_pred = (y_prob >= 0.5).astype(int)

acc  = accuracy_score(y_val, y_pred)
prec, rec, f1, _ = precision_recall_fscore_support(
    y_val, y_pred, labels=[0, 1], zero_division=0
)

print("\n──── Validation metrics (thr = 0.50) ────")
print(f"Accuracy          : {acc:6.3f}")
print(f"Class 0 (Down) →  Precision: {prec[0]:6.3f}  Recall: {rec[0]:6.3f}  F1: {f1[0]:6.3f}")
print(f"Class 1 (Up  ) →  Precision: {prec[1]:6.3f}  Recall: {rec[1]:6.3f}  F1: {f1[1]:6.3f}")
print(f"Macro-F1          : {f1.mean():6.3f}")

# optional: clean tuner directory
shutil.rmtree(TMP_DIR, ignore_errors=True)




──── Best hyper-parameters ────
conv_blocks : 2
filters_0   : 32
kernel_0    : 3
dropout     : 0.0
lr          : 0.00013468371522638964
filters_1   : 32
kernel_1    : 3
Epoch 1/30
1008/1008 - 8s - 7ms/step - loss: 0.6947 - val_loss: 0.9206 - val_weighted_f1: 0.6533 - weighted_f1: 0.4944
Epoch 2/30
1008/1008 - 6s - 6ms/step - loss: 0.6921 - val_loss: 0.7637 - val_weighted_f1: 0.1995 - weighted_f1: 0.5108
Epoch 3/30
1008/1008 - 6s - 6ms/step - loss: 0.6911 - val_loss: 0.9273 - val_weighted_f1: 0.1063 - weighted_f1: 0.5117
Epoch 4/30
1008/1008 - 6s - 6ms/step - loss: 0.6904 - val_loss: 1.0318 - val_weighted_f1: 0.0884 - weighted_f1: 0.5115
Epoch 5/30
1008/1008 - 6s - 6ms/step - loss: 0.6898 - val_loss: 1.0748 - val_weighted_f1: 0.0868 - weighted_f1: 0.5112
Epoch 6/30
1008/1008 - 3s - 3ms/step - loss: 0.6893 - val_loss: 1.1608 - val_weighted_f1: 0.0752 - weighted_f1: 0.5111
Epoch 7/30
1008/1008 - 3s - 3ms/step - loss: 0.6889 - val_loss: 1.2164 - val_weighted_f1: 0.0721 - weighted_f1: 0.5

In [10]:
"""
Train a 1-D CNN for BTC next-hour direction
------------------------------------------
Uses your selected hyper-parameters (see header).
Outputs per-class precision, recall, F1 and overall accuracy.
"""

import numpy as np, pandas as pd, tensorflow as tf
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

# ─────────── File paths & constants ───────────
CSV_PATH  = r"C:\Users\ADMIN\Desktop\Coding_projects\stock_market_prediction\Stock-Market-Prediction\data\processed\gemini_btc_data_final_version_with_features_2016_final.csv"
DROP_COLS = ["vol_ratio_24h", "macd_diff", "macd_line", "upper_shadow", "lower_shadow"]

SEQ_LEN   = 60          # past 60 hours
VAL_FRAC  = 0.20        # last 20 % → validation
W_PREC    = 2.0         # precision weight in weighted-F1
LR        = 1.3468371522638964e-4
MAX_EPOCH = 30
BATCH     = 64

# ─────────── Weighted-F1 (precision×2) ───────────
class WeightedF1(tf.keras.metrics.Metric):
    def __init__(self, weight=2.0, name="weighted_f1", threshold=0.5, **kw):
        super().__init__(name=name, **kw)
        self.w  = weight
        self.th = threshold
        self.tp = self.add_weight(name="tp", initializer="zeros")
        self.fp = self.add_weight(name="fp", initializer="zeros")
        self.fn = self.add_weight(name="fn", initializer="zeros")

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_pred = tf.cast(y_pred >= self.th, tf.float32)
        y_true = tf.cast(y_true, tf.float32)
        self.tp.assign_add(tf.reduce_sum(y_true * y_pred))
        self.fp.assign_add(tf.reduce_sum((1 - y_true) * y_pred))
        self.fn.assign_add(tf.reduce_sum(y_true * (1 - y_pred)))

    def result(self):
        prec = self.tp / (self.tp + self.fp + 1e-7)
        rec  = self.tp / (self.tp + self.fn + 1e-7)
        return (1 + self.w) * prec * rec / (self.w * prec + rec + 1e-7)

    def reset_states(self):
        for v in (self.tp, self.fp, self.fn):
            v.assign(0.)

# ─────────── Data loading & preprocessing ───────────
df = pd.read_csv(CSV_PATH, index_col=0, parse_dates=True)
df = df.drop(columns=[c for c in DROP_COLS if c in df.columns])
df["Volume BTC"] = np.log1p(df["Volume BTC"])

# label: next close up (1) / down (0)
df["target"] = (df["close"].shift(-1) > df["close"]).astype(int)
df = df.dropna().select_dtypes(include=[np.number])

feature_cols = df.columns.drop("target")
split_raw = int(len(df) * (1 - VAL_FRAC))
train_raw, val_raw = df.iloc[:split_raw], df.iloc[split_raw:]

scaler = StandardScaler().fit(train_raw[feature_cols])
df_scaled = pd.DataFrame(
    scaler.transform(df[feature_cols]),
    columns=feature_cols, index=df.index
)
labels = df["target"].values.astype(np.float32)

def make_sequences(mat, tgt, length):
    Xs, ys = [], []
    for i in range(length, len(mat)):
        Xs.append(mat[i-length:i])
        ys.append(tgt[i])
    return np.array(Xs, dtype=np.float32), np.array(ys, dtype=np.float32)

X_all, y_all = make_sequences(df_scaled.values, labels, SEQ_LEN)
split_seq = int(len(X_all) * (1 - VAL_FRAC))
X_train, X_val = X_all[:split_seq], X_all[split_seq:]
y_train, y_val = y_all[:split_seq], y_all[split_seq:]
n_features = X_train.shape[2]

# ─────────── Build CNN with fixed hyper-params ───────────
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(SEQ_LEN, n_features)),
    tf.keras.layers.Conv1D(32, 3, padding="causal", activation="relu"),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv1D(32, 3, padding="causal", activation="relu"),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dropout(0.0),      # as specified
    tf.keras.layers.Dense(1, activation="sigmoid")
])

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=LR),
    loss="binary_crossentropy",
    metrics=[WeightedF1(weight=W_PREC)]
)

early_stop = tf.keras.callbacks.EarlyStopping(
    patience=5, restore_best_weights=True, monitor="val_loss"
)

# ─────────── Train ───────────
model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=MAX_EPOCH,
    batch_size=BATCH,
    shuffle=False,
    callbacks=[early_stop],
    verbose=2
)

# ─────────── Evaluate ───────────
y_prob = model.predict(X_val, batch_size=BATCH).flatten()
y_pred = (y_prob >= 0.5).astype(int)

acc  = accuracy_score(y_val, y_pred)
prec, rec, f1, _ = precision_recall_fscore_support(
    y_val, y_pred, labels=[0, 1], zero_division=0
)

print("\n──── Validation metrics (thr = 0.50) ────")
print(f"Accuracy          : {acc:6.3f}")
print(f"Class 0 (Down) →  Precision: {prec[0]:6.3f}  Recall: {rec[0]:6.3f}  F1: {f1[0]:6.3f}")
print(f"Class 1 (Up  ) →  Precision: {prec[1]:6.3f}  Recall: {rec[1]:6.3f}  F1: {f1[1]:6.3f}")
print(f"Macro-F1          : {f1.mean():6.3f}")


Epoch 1/30
1008/1008 - 4s - 4ms/step - loss: 0.6935 - val_loss: 0.7591 - val_weighted_f1: 0.3773 - weighted_f1: 0.5113
Epoch 2/30
1008/1008 - 3s - 3ms/step - loss: 0.6915 - val_loss: 0.7899 - val_weighted_f1: 0.2776 - weighted_f1: 0.5193
Epoch 3/30
1008/1008 - 3s - 3ms/step - loss: 0.6907 - val_loss: 0.8129 - val_weighted_f1: 0.2425 - weighted_f1: 0.5180
Epoch 4/30
1008/1008 - 3s - 3ms/step - loss: 0.6901 - val_loss: 0.8438 - val_weighted_f1: 0.2063 - weighted_f1: 0.5172
Epoch 5/30
1008/1008 - 3s - 3ms/step - loss: 0.6896 - val_loss: 0.8955 - val_weighted_f1: 0.1615 - weighted_f1: 0.5172
Epoch 6/30
1008/1008 - 3s - 3ms/step - loss: 0.6891 - val_loss: 0.9584 - val_weighted_f1: 0.1347 - weighted_f1: 0.5164
252/252 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step

──── Validation metrics (thr = 0.50) ────
Accuracy          :  0.497
Class 0 (Down) →  Precision:  0.490  Recall:  0.666  F1:  0.565
Class 1 (Up  ) →  Precision:  0.511  Recall:  0.335  F1:  0.405
Macro-F1          :  0.485
